## MLflow's Model Registry

In [62]:
%env MLFLOW_TRACKING_URI=http://localhost:5000
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ACCESS_KEY_ID=2vSrPs21nZYaUQvovgRL
%env AWS_SECRET_ACCESS_KEY=yCqF29KU1qbykEnsceWMDRNvPelgGAVBmyD6PeU5

env: MLFLOW_TRACKING_URI=http://localhost:5000
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ACCESS_KEY_ID=2vSrPs21nZYaUQvovgRL
env: AWS_SECRET_ACCESS_KEY=yCqF29KU1qbykEnsceWMDRNvPelgGAVBmyD6PeU5


In [63]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "http://localhost:5000"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [64]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.search_experiments()

[<Experiment: artifact_location='s3://mlflow/2', creation_time=1701007317661, experiment_id='2', last_update_time=1701007317661, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='s3://mlflow/1', creation_time=1700794101778, experiment_id='1', last_update_time=1700794101778, lifecycle_stage='active', name='nyc-taxi', tags={}>,
 <Experiment: artifact_location='s3://mlflow/0', creation_time=1700747453840, experiment_id='0', last_update_time=1700817617764, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
client.create_experiment(name="my-cool-experiment")

Let's check the latest versions for the experiment with id `1`...

In [65]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="""metrics."validation-rmse" < 8""",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["""metrics."validation-rmse" ASC"""],
)

In [66]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['validation-rmse']:.4f}")

run id: 8817e578f29b43598346b54eea60617d, rmse: 6.4011
run id: 92189bb33c63437c987322e18db45adf, rmse: 6.4011
run id: b645a1faa8b04a2287b5ddaecf9fd4a0, rmse: 6.5711


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `5` to "Staging" and adding annotations to it.

In [67]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [68]:
run_id = "8817e578f29b43598346b54eea60617d"
# model_uri = f"runs:/{run_id}/model"  # awalnya ini yang buat error, karena uri tidak ditemukan
model_uri = f"runs:/{run_id}/models"  # hati-hati dalam mengambil model_uri (bisa dilihat di minio s3 dulu)
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/11/26 21:56:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 5
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1701010571927, current_stage='None', description='', last_updated_timestamp=1701010571927, name='nyc-taxi-regressor', run_id='8817e578f29b43598346b54eea60617d', run_link='', source='s3://mlflow/1/8817e578f29b43598346b54eea60617d/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='5'>

In [69]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 5, stage: None


In [70]:
model_version = 5
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1701010571927, current_stage='Staging', description='', last_updated_timestamp=1701010579275, name='nyc-taxi-regressor', run_id='8817e578f29b43598346b54eea60617d', run_link='', source='s3://mlflow/1/8817e578f29b43598346b54eea60617d/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='5'>

In [71]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1701010571927, current_stage='Staging', description='The model version 5 was transitioned to Staging on 2023-11-26', last_updated_timestamp=1701010581686, name='nyc-taxi-regressor', run_id='8817e578f29b43598346b54eea60617d', run_link='', source='s3://mlflow/1/8817e578f29b43598346b54eea60617d/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='5'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [72]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [73]:
df = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [74]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/sugab/workspace/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [75]:
import pickle

with open("./preprocessor/preprocessor.bin", "rb") as f_in:
    dv = pickle.load(f_in)

In [76]:
X_test = preprocess(df, dv)

In [77]:
target = "duration"
y_test = df[target].values

In [78]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/sugab/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:56:34] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 936 ms, sys: 9.93 ms, total: 946 ms
Wall time: 675 ms


{'rmse': 6.571139749510483}

In [80]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

/home/sugab/miniconda3/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:56:41] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 3.36 s, sys: 19.3 ms, total: 3.38 s
Wall time: 1.87 s


{'rmse': 6.4011367052953405}

Dari perbanding performa model registry pada stage production dengan staging diatas, maka diambil model registry pada staging stage karena rmse lebih rendah walaupun waktu komputasinya lebih lama sedikit.

Kemudian ubah model registry pada staging menjadi production dan archive model production sebelumnya.

In [81]:
client.transition_model_version_stage(
    name=model_name,
    version=5,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1701010571927, current_stage='Production', description='The model version 5 was transitioned to Staging on 2023-11-26', last_updated_timestamp=1701010611956, name='nyc-taxi-regressor', run_id='8817e578f29b43598346b54eea60617d', run_link='', source='s3://mlflow/1/8817e578f29b43598346b54eea60617d/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='5'>

## What i learned

* Secara umum, stage pada model registry yaitu:
  * None: belum dilabeli stage-nya
  * Archive: model sudah tidak dipakai
  * Staging: model sedang dikembangkan
  * Production: model siap dideploy
* Model registry ini berguna untuk mendaftarkan model dari tracking server ke model yang telah dilabeli versi-nya atau stage-nya
* Sehingga manajemen model dapat lebih mudah
* Untuk melakukan automation kedepannya, kita dapat menggunakan MLflow client pada python sehingga tidak perlu berinteraksi pada UI-nya
  * Ini mungkin akan sangat terpakai waktu CI/CD atau orchestration nanti